<a href="https://colab.research.google.com/github/farnaz-frd/Closest_sentences-from-different-languages-with-laser-embedding/blob/main/top_K_closest_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laser embedding with cosine similarity

### Installing require liberaries

In [3]:
pip install laserembeddings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached laserembeddings-1.1.2-py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 55.7 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=f8cc5889fd4f6871ca9f3cbe3070396bb37be6abbaa8ea780e05bd0b152ce751
  Stored in directory: /root/.cache/pip/wheels/8e/12/4b/5c9eeed3636a4041

In [4]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [5]:
ls

de.unpar.txt  pt.unpar.txt  sample_data/


### Main part

In [6]:
import torch
from laserembeddings import Laser

laser = Laser()

# Load the sentences from the files
with open('de.unpar.txt', 'r') as f:
    de_sentences = f.read().splitlines()
with open('pt.unpar.txt', 'r') as f:
    pt_sentences = f.read().splitlines()

# Compute the embeddings
de_embeddings = laser.embed_sentences(de_sentences, lang='de')
pt_embeddings = laser.embed_sentences(pt_sentences, lang='pt')

# Compute the cosine similarity
similarity = torch.cosine_similarity(torch.tensor(de_embeddings).unsqueeze(1), torch.tensor(pt_embeddings).unsqueeze(0), dim=2)

# Get the top K similar sentences
K = 100
top_K_indices = torch.topk(similarity.flatten(), K).indices
top_K_indices = top_K_indices.tolist()
top_K_indices = [(i // similarity.shape[1], i % similarity.shape[1]) for i in top_K_indices]

# Write the results to a file
with open('output.txt', 'w') as f:
    for i, j in top_K_indices:
        f.write(f'{de_sentences[i]}\t{pt_sentences[j]}\n')

# Laser embedding with The approximate nearest neighbor search algorithm

### Installing require liberaries

In [8]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 36.1 MB/s eta 0:00:00


### Main part


In [9]:
import faiss
import numpy as np
from laserembeddings import Laser

laser = Laser()

# Load the sentences from the files
with open('de.unpar.txt', 'r') as f:
    de_sentences = f.read().splitlines()
with open('pt.unpar.txt', 'r') as f:
    pt_sentences = f.read().splitlines()

# Compute the embeddings
de_embeddings = laser.embed_sentences(de_sentences, lang='de')
pt_embeddings = laser.embed_sentences(pt_sentences, lang='pt')

# Normalize the embeddings
de_embeddings = de_embeddings / np.linalg.norm(de_embeddings, axis=1, keepdims=True)
pt_embeddings = pt_embeddings / np.linalg.norm(pt_embeddings, axis=1, keepdims=True)

# Build a faiss index
d = de_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(pt_embeddings.astype('float32'))

# Find the top k most similar pairs of sentences
K = 100
D, I = index.search(de_embeddings.astype('float32'), K)

# Write the results to a file
with open('output2.txt', 'w') as f:
    for i in range(K):
        f.write(f'{de_sentences[I[i][0]]}\t{pt_sentences[I[i][0]]}\n')